### 1. 라이브러리 선언

In [1]:
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
import csv
import urllib.request
import urllib.parse
import bs4
import pandas as pd

### 2. 데이터 불러오기

### 3. 사이트 호출

In [2]:
driver_loc = "../externlib/chromedriver/chromedriver.exe"

options = webdriver.ChromeOptions()

options.add_argument("window-size=1920x1080")

driver = webdriver.Chrome(driver_loc, options=options)

baseUrl = "https://www.mangoplate.com/"

driver.get(baseUrl)

driver.current_url

C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/3732077489.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_loc, options=options)


'https://www.mangoplate.com/'

In [3]:
closePath = '/html/body/div[14]'
driver.find_element_by_xpath(closePath).click()

C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/3754975954.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(closePath).click()


In [4]:
# 검색 데이터 만들기

sinsaAddress = "강남구 신사동"

In [6]:
driver.switch_to.window(driver.window_handles[1])

In [7]:
driver.close()

In [8]:
driver.switch_to.window(driver.window_handles[0])

In [9]:
# 검색어 입력(for문 돌려야됨)
searchPath = '//*[@id="main-search"]'
driver.find_element_by_xpath(searchPath).send_keys(sinsaAddress)

C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/3505454210.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).send_keys(sinsaAddress)


In [10]:
driver.find_element_by_xpath(searchPath).send_keys(Keys.ENTER)

C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/2100832940.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).send_keys(Keys.ENTER)


In [13]:
# 빈 리스트 생성
cafeName = [] # 식당이름
foodScore = [] # 별점
reviewCount = [] # 리뷰
likeCount = [] # 좋아요수
address = [] # 주소
phoneNumber = [] # 전화번호
foodKategorie = [] # 식당 카테고리
parking = [] # 주차
openHour = [] # 영업시간
menuList = [] # 메뉴
menuAllList = []
reviewsList = [] # 리뷰

In [17]:
for l in range(1,11):
    html = driver.page_source
    ## BeautifulSoup로 페이지 소스 파싱
    bs = bs4.BeautifulSoup(html, "html.parser")
    
    time.sleep(3)
    
    pageNumber = '/html/body/main/article/div[2]/div/div/section/div[4]/p/a[pageNum]'
    pageCheck = pageNumber.replace('pageNum',str(l))
    driver.find_element_by_xpath(pageCheck).click()
    
    for k in range(1,11) : ### test 중 11 까지 다시 변경하기
        # 식당 선택 페이지 이동
        time.sleep(3)
        cafeClick = '/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[row]/div[col]/figure/a'
        clickRow = cafeClick.replace('row',str(k))

        for j in range(1,3):

            clikckInfoNum = clickRow.replace('col',str(j))
            clikckInfoNum
            driver.find_element_by_xpath(clikckInfoNum).click()

            time.sleep(2)

            html = driver.page_source
            html

            ## BeautifulSoup로 페이지 소스 파싱
            bs = bs4.BeautifulSoup(html, "html.parser")

            bs
            time.sleep(2)
            try: 
                try :
                # 식당이름
                    cafeCheck = bs.find(attrs={"class":"restaurant_name"} ).text
                    cafeName.append(cafeCheck)

                except :
                    cafeName.append("none")


                try :            
                # 별점
                    foodScoreCheck = bs.find(attrs={"class":"rate-point"} ).text.replace("\n","")
                    foodScoreCheck

                    foodScore.append(foodScoreCheck)


                except:
                    foodScore.append("none")

                try:
                    # 리뷰수
                    reviewCheck = bs.find(attrs={"class":"cnt review"} ).text
                    reviewCount.append(reviewCheck)


                except:

                    reviewCount.append("none")



                try:
                    # 좋아요 수
                    likeCheck = bs.find(attrs={"class":"cnt favorite"} ).text
                    likeCount.append(likeCheck)
                    likeCount


                except:
                    likeCount.append("none")



                try:
                    # 주소
                    Info = bs.find(attrs={"class":"info"}).text

                    Info.split("\n")

                    Info.split("\n").index("주소")

                    addressCheck = Info.split("\n")[Info.split("\n").index("주소")+1]
                    addressCheck

                    address.append(addressCheck)

                    address


                except:
                    address.append("none")

                try:
                    # 전화번호
                    phoneCheck = Info.split("\n")[Info.split("\n").index("전화번호")+1]
                    phoneCheck

                    phoneNumber.append(phoneCheck)

                    phoneNumber


                except:
                    phoneNumber.append("none")





                try:
                    # 식당 카테고리
                    Info.split("\n")[Info.split("\n").index("음식 종류")+2]

                    foodCheck = Info.split("\n")[Info.split("\n").index("음식 종류")+2]
                    foodKategorie.append(foodCheck)


                except:
                    foodKategorie.append("")



                try:
                    # 주차
                    parkCheck = Info.split("\n")[Info.split("\n").index("주차")+1]
                    parking.append(parkCheck)

                except:
                    parking.append("none")




                try:
                    # 영업시간 (이거 확인 다 해봐야 됨)
                    openCheck = Info.split("\n")[Info.split("\n").index("영업시간")+1]
                    openHour.append(openCheck)


                except:
                    openHour.append("none")




                try:
                    # 메뉴
                    menuAll = ""
                    # 메뉴 5개
                    for i in range(2,12):
                        menuCheck = Info.split("\n")[Info.split("\n").index("메뉴")+(2*i)] + "_" + Info.split("\n")[Info.split("\n").index("메뉴")+(2*i)+1] + "|"
                        menuList.append(menuCheck)
                    menuList

                    len(menuList)

                    menuList.index("_|")

                    for i in range(0,len(menuList)):
                        if ("_|" in menuList) == True :
                            menuList.remove("_|")
                        else :
                            pass


                    for i in range(0,len(menuList)):
                        menuAll += menuList[i]
                    menuAllList.append(menuAll)
                    menuAllList

                except:

                    menuAllList.append("none")





                try:
                    # 리뷰 3개

                    reviewCheck = bs.findAll(attrs={"class":"RestaurantReviewItem__ReviewText"} )
                    reviewCheck[0].text.split("\n")[1]

                    reviews = ""
                    for i in range(0,3):
                        reviews += reviewCheck[i].text.split("\n")[1] + "|"
                    reviewsList.append(reviews)

                    # 페이지 뒤로가기
                    time.sleep(2)
                    driver.back()
                except:
                    reviewsList.append("none")

                    # 페이지 뒤로가기
                    time.sleep(2)
                    driver.back()
            except:
                pass

C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/743202205.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(pageCheck).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_21020/743202205.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()


In [18]:
cafeNameDf = pd.DataFrame(cafeName, columns=["가게명"])
foodScoreDf = pd.DataFrame(foodScore, columns=["별점"])
reviewCountDf = pd.DataFrame(reviewCount, columns=["리뷰수"])
likeCountDf = pd.DataFrame(likeCount, columns=["좋아요수"])
addressDf = pd.DataFrame(address, columns=["주소"])
phoneNumberDf = pd.DataFrame(phoneNumber, columns=["전화번호"])
foodKategorieDf = pd.DataFrame(foodKategorie, columns=["카테고리"])
parkingDf = pd.DataFrame(parking, columns=["주차"])
openHourDf = pd.DataFrame(openHour, columns=["영업시간"])
menuAllListDf = pd.DataFrame(menuAllList, columns=["메뉴"])
reviewsListDf = pd.DataFrame(reviewsList, columns=["리뷰"])

In [19]:
result = pd.concat([cafeNameDf,foodScoreDf,reviewCountDf,likeCountDf,addressDf,phoneNumberDf,foodKategorieDf,parkingDf,openHourDf,menuAllListDf,reviewsListDf],axis=1)

In [20]:
result

,가게명,별점,리뷰수,좋아요수,주소,전화번호,카테고리,주차,영업시간,메뉴,리뷰
0,미라이,4.8,120,"9,400",서울특별시 강남구 논현로153길 24 끌레르빌 1F,02-540-0934,이자카야 / 오뎅 / 꼬치,주차공간없음,18:00 - 01:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",사바산도….. 예약하지 않으면 갈 수 없는 이자카야다.주말에 갔는...
1,시라카와,4.8,66,"5,486",서울특별시 강남구 선릉로161길 7 1F,070-4115-4769,이자카야 / 오뎅 / 꼬치,발렛,17:00 - 23:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",서울특별시 강남구 선릉로| 튀김은 웬만하면 다 ...
2,일리조,4.8,38,"1,380",서울특별시 강남구 압구정로42길 23 1F,02-544-1980,이탈리안,유료주차 가능,11:30 - 22:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",같이 간 친구가 간 압구정에 괜찮은 파스타 집이 있다고 해서 들렀...
3,로바타탄요,4.7,91,"6,410",서울특별시 강남구 압구정로28길 49 109호,02-6339-9667,정통 일식 / 일반 일식,발렛,17:00 - 22:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...","** 자리에서 하나씩 구워주는 야키토리, 맛없을 수 없죠이미 너무..."
4,코슌,4.7,23,"1,166",서울특별시 강남구 언주로153길 14 2F,0507-1315-8729,이자카야 / 오뎅 / 꼬치,발렛,17:00 - 24:00,none,❤︎맛있지만 좀 더 야키토리 본연의 맛을 느끼고 싶었던 곳퇴근 후...
...,...,...,...,...,...,...,...,...,...,...,...
195,누데이크,4.0,25,524,서울특별시 강남구 압구정로46길 50 하우스도산 B1,070-4128-2125,카페 / 디저트,"발렛 3,000원",11:00 - 21:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",나도 드디어 다슬기 크로와상 먹어봤다 !! 에어팟만한 종이가방에 ...
196,코타by뎐,4.0,29,"2,321",서울특별시 강남구 선릉로155길 23-3,02-3445-5577,일반 주점,"발렛 2시간 3,000원",18:00 - 24:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",코 타바이뎐과 함께한 한식코스요리타 노스급의 맛과 비쥬얼은 날 어...
197,크라운돼지,4.0,25,443,서울특별시 강남구 강남대로156길 17-1,02-512-5700,고기 요리,발렛,월-토: 17:00 - 24:00 일: 17:00 - 22:00,none,목살을 먹기 위해서라도 가야한다.가로수길에 있는 돼지고기집이다.신...
198,플디,4.0,28,758,서울특별시 강남구 압구정로42길 24-5 로비스빌딩 4F,010-2802-6341,카페 / 디저트,"발렛 3,000원",12:00 - 22:00,"사시미8종 (2인)_41,000원|마구로치즈아에_19,000원|양갈비 스미비 야끼_...",압구정 로데오에 위치한 카페!평일 점심시간이 지나고 방문하니 손님...


In [21]:
fileName = '../dataset/manggoplate.csv'
result.to_csv(fileName, index=False, encoding="utf-8-sig")